In [1]:
import pandas as pd
%load_ext autoreload
%autoreload 2

In [2]:
import django
import os

os.environ['DJANGO_SETTINGS_MODULE'] = 'dashboard.settings'
os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"
django.setup()

In [4]:
import pandas as pd

In [28]:
walklist = pd.read_csv('../resources/hd51/hd51_walklist.csv', dtype=str,
                       engine='python', keep_default_na=False)

In [29]:
walklist = walklist.assign(voter_id=walklist.A1.apply(lambda x: f'{int(x):08d}'))
walklist.head()

,A1,Contacted,Supporter,Yardisgn,County,CountyName,LAST_NAME,FIRST_NAME,MIDDLE_MAIDEN_NAME,NAME_SUFFIX,...,MUNOV21,MURUNOFF21,SPEC22,Pri22,Pri22abs,Prirunoff2022,Prirunoff2022ABS,RegistrationAddressLatitude,RegistrationAddressLongitude,voter_id
0,99615,,,,60,FULTON,DOWLING,KATHERINE,KRAUSE,,...,,,,,,,,33.926907,-84.360377,00099615
1,115151,,,,60,FULTON,YEAGER,HARMON,TODD,,...,,,,,,,,33.961134,-84.349388,00115151
2,117807,,,,60,FULTON,DAHLEN,STEVEN,JOHN,,...,,,,,,,,33.995456,-84.287766,00117807
3,121602,,,,60,FULTON,CARDEN,AMY,STAATS,,...,,,,,,,,33.997614,-84.307868,00121602
4,145174,,,,60,FULTON,SHOEMAKER,SANDRA,L,,...,A,,,R,A,R,Y,34.051875,-84.387688,00145174


In [30]:
from voter.models import Voter, ListEdition, ResidenceAddress
from datetime import datetime

In [ ]:
edition_date = datetime.strptime('2022-8-5', '%Y-%m-%d')
edition = ListEdition.objects.get(date=edition_date)
records = []
for i in walklist.index:
    row = walklist.loc[i]
    v = Voter.objects.filter(voter_id=row.voter_id, edition=edition).first()
    if v is None or v.hse != '051':
        continue
    a = v.residence_address_of.first()
    records.append(
        {
            'street': f'{a.house_number} {a.street_name}',
            'city': a.city,
            'state': 'GA',
            'zipcode': a.zipcode
        }
    )
df = pd.DataFrame.from_records(records).drop_duplicates()
df.shape

In [32]:
df = df.reset_index(drop=True)

In [37]:
df1000 = df.head(1000)

In [38]:
df1000.to_csv('../resources/hd51/geocode_upload1.csv', index=False)

In [33]:
dfrest = df[1000:]

In [ ]:
dfrest.shape

In [35]:
dfrest.to_csv('../resources/hd51/geocode_upload_rest.csv', index=False)

In [3]:
from walklist.models import Walklist
from hse_map.models import HseMap
from datetime import datetime
from voter.models import ListEdition

In [4]:
edition_date = datetime.strptime('2022-08-05', '%Y-%m-%d')
edition = ListEdition.objects.get(date=edition_date)
hd51 = HseMap.objects.get(district='051')
hd51.edition = edition

In [38]:
walklist = Walklist(hd51)



In [ ]:
d.head(20)

In [ ]:
m = walklist.build_address_score_map()

In [ ]:
m.show()